In [2]:
# Download ERMA from github
PATH="/local/work/adrian/test"
!mkdir -p ${PATH}
!git clone https://github.com/IKIM-Essen/ERMA.git $(PATH)


/bin/bash: PATH: command not found
/bin/bash: github: command not found
